<a href="https://colab.research.google.com/github/AmulyaThammineni/6440304_Thammineni_Naga_Amulya/blob/main/mydemo1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio transformers torch torchaudio librosa ffmpeg-python moviepy gtts git+https://github.com/openai/whisper.git --quiet
!apt-get install ffmpeg -y


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 7.2 MB/s eta

In [ ]:
!pip uninstall -y whisper


In [ ]:
!pip install -q git+https://github.com/openai/whisper.git


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install gTTS

In [ ]:
from gtts import gTTS


In [ ]:
pip install pyttsx3

In [ ]:
import os
import torch
import librosa
import whisper
import gradio as gr
from transformers import AutoTokenizer, AutoModelForMaskedLM
from moviepy.editor import VideoFileClip, AudioFileClip
import pyttsx3


  if event.key is 'enter':



In [ ]:
def extract_audio(video_path, output_audio_path="audio.wav"):
    video = VideoFileClip(video_path)
    video.audio.write_audiofile(output_audio_path)
    return output_audio_path


In [ ]:
def detect_silent_parts(audio_path, silence_db=35):
    y, sr = librosa.load(audio_path)
    intervals = librosa.effects.split(y, top_db=silence_db)

    silent_parts = []
    last_end = 0
    for start, end in intervals:
        if start > last_end:
            silent_parts.append((last_end / sr, start / sr))
        last_end = end
    return silent_parts


In [ ]:
def transcribe_audio(audio_path):
    model = whisper.load_model("base")
    result = model.transcribe(audio_path)
    return result["text"]


In [ ]:
def fill_mask(text_with_masks, model_name="bert-base-uncased", top_k=1):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForMaskedLM.from_pretrained(model_name)

    inputs = tokenizer.encode_plus(text_with_masks, return_tensors="pt")
    mask_indices = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]

    with torch.no_grad():
        logits = model(**inputs).logits

    for idx in mask_indices:
        probs = logits[0, idx]
        top_preds = torch.topk(probs, top_k).indices
        word = tokenizer.decode(top_preds[0])
        text_with_masks = text_with_masks.replace("[MASK]", word.strip(), 1)

    return text_with_masks


In [ ]:
def text_to_speech(text, out_path="synth_audio.wav"):
    tts = gTTS(text)
    tts.save(out_path)
    return out_path


In [ ]:
def combine_audio_video(video_path, audio_path, output_path="final_output.mp4"):
    video = VideoFileClip(video_path)
    new_audio = AudioFileClip(audio_path)
    final = video.set_audio(new_audio)
    final.write_videofile(output_path, codec="libx264", audio_codec="aac")
    return output_path


In [ ]:
def process_video(video_file):
    try:
        video_path = video_file.name if hasattr(video_file, "name") else video_file

        # 1. Extract audio
        audio_path = extract_audio(video_path)

        # 2. Detect silent parts
        silence_segments = detect_silent_parts(audio_path)

        # 3. Transcribe
        transcript = transcribe_audio(audio_path)

        # 4. Add [MASK]s for silence parts
        masked_text = transcript + " " + " ".join(["[MASK]"] * len(silence_segments))

        # 5. Predict missing words
        reconstructed_text = fill_mask(masked_text)

        # 6. Convert to speech
        synth_audio_path = text_to_speech(reconstructed_text)

        # 7. Combine with original video
        final_video = combine_audio_video(video_path, synth_audio_path)

        return final_video, transcript, masked_text, reconstructed_text
    except Exception as e:
        return None, f"Error: {str(e)}", "Error", "Error"


In [ ]:
gr.Interface(
    fn=process_video,
    inputs=gr.Video(label="🎥 Upload video with muted parts"),
    outputs=[
        gr.Video(label="🎬 Reconstructed Video"),
        gr.Textbox(label="📝 Original Transcript"),
        gr.Textbox(label="🧩 Masked Transcript"),
        gr.Textbox(label="✅ Reconstructed Transcript")
    ],
    title="🔊 AI Video Speech Reconstructor",
    description="Upload a muted video. This tool predicts missing speech using BERT and reconstructs it with audio.",
    allow_flagging="never"
).launch()


  warnings.warn(



It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7b916a7d2500a5bc00.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
